In [1]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import json

In [2]:
POPULATION_INP_ROOT = '/u/ayushc/projects/GradABM/MacroEcon/simulator_data/census_populations/NYC/user_input/'

In [3]:
pop_data_file = os.path.join(POPULATION_INP_ROOT, 'NYC_POP.pkl')

In [4]:
from census_loader import CensusDataLoader

/u/ayushc/agent_torch_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-17 15:59:50,182	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
pop_data_df = pd.read_pickle(pop_data_file)
area_selector = pop_data_df['age_gender']['area'][:1]
geo_mapping = None

In [6]:
# df_age_gender_combined = pop_data_df['age_gender']
# df = df_age_gender_combined[df_age_gender_combined['area']=='QN0103']
# area_selector = df['area'][:1]
# area_selector
# area_selector = ['BK0101']

In [6]:
loader = CensusDataLoader(n_cpu=8, use_parallel=False, area_list=area_selector, geo_mapping=geo_mapping)

In [7]:
POPULATION_OUT_ROOT = '/u/ayushc/projects/GradABM/MacroEcon/simulator_data/census_populations/NYC/{}_synthetic_populations/'.format(area_selector[0])

if not os.path.isdir(POPULATION_OUT_ROOT):
    os.makedirs(POPULATION_OUT_ROOT)

# Create Synthetic Agent Population

In [8]:
save_population_data_path = os.path.join(POPULATION_OUT_ROOT, 'population_data.pkl')
loader.generate_basepop(input_data=pop_data_df, save_path=save_population_data_path)

INFO:process.base_pop:Processing: 0/1


/u/ayushc/projects/GradABM/MacroEcon/simulator_data/process/base_pop.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ethnicity_data['probability'] = ethnicity_data.apply(lambda row: row['count'] / total_population[row['area']], axis=1)
/u/ayushc/projects/GradABM/MacroEcon/simulator_data/process/base_pop.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ethnicity_data['probability'] = ethnicity_data.apply(lambda row: row['count'] / total_population[row['area']], axis=1)
/u/ayushc/projects/GradABM/Mac

In [9]:
loader.population_df.columns

Index(['area', 'age', 'gender', 'ethnicity'], dtype='object')

In [10]:
loader.population_df['area'].shape

(37518,)

## Create Synthetic Mobility Networks

In [11]:
num_steps = 2
mobility_mapping_path = os.path.join(POPULATION_INP_ROOT, 'mobility_mapping.json')

print(os.path.exists(mobility_mapping_path))

with open(mobility_mapping_path) as file:
    mobility_mapping = json.load(file)
    
save_mobility_network_dir = os.path.join(POPULATION_OUT_ROOT, 'mobility_networks/')
print("Mobility Network Dir: ", save_mobility_network_dir)
    
loader.generate_mobility_networks(num_steps, mobility_mapping, save_mobility_network_dir)

True
Mobility Network Dir:  /u/ayushc/projects/GradABM/MacroEcon/simulator_data/census_populations/NYC/BK0101_synthetic_populations/mobility_networks/


## Generate Household Population

In [13]:
household_data_path = os.path.join(POPULATION_INP_ROOT, 'household.pkl')
#'../../housing_v2.pkl'
household_data = np.load(household_data_path, allow_pickle=True)
household_mapping = {'adult_list': ['20t29','30t39', '40t49', '50t64', '65A'], 'children_list': ['U19']}

In [14]:
household_data['household_num']

0      17487
1      26955
2      10613
3      22665
4      11857
       ...  
257        0
258        9
259        0
260       66
261        0
Name: household_num, Length: 262, dtype: int64

In [15]:
save_household_path = os.path.join(POPULATION_OUT_ROOT, 'base_household.pkl')
loader.generate_household(household_data, household_mapping, save_path=save_household_path)

INFO:process.household:0/1: Processing QN0103
/u/ayushc/projects/GradABM/MacroEcon/simulator_data/process/household.py:377: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_household_data["adult_num"] = (
/u/ayushc/projects/GradABM/MacroEcon/simulator_data/process/household.py:381: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_household_data["adult_num"] = proc_household_data["adult_num"].clip(lower=0)
/u/ayushc/projects/GradABM/MacroEcon/simulator_data/process/household.py:196: SettingWithCopyWarnin

TypeError: 'Series' object cannot be interpreted as an integer

## Generate demographic data files

In [13]:
# def preprocess_data(df_path, output_dir):
#     df = pd.read_pickle(df_path)
#     attributes = df.keys()
#     mapping_collection = {}
#     for attribute in attributes:
#         df[attribute],mapping = pd.factorize(df[attribute])
#         output_att_path = os.path.join(output_dir, attribute)
#         df[attribute].to_pickle(f'{output_att_path}.pickle')
#         mapping_collection[attribute] = mapping.tolist()
#     output_mapping_path = os.path.join(output_dir, 'population_mapping.json')
    
#     print("output mapping path: ", output_mapping_path)
    
#     with open(output_mapping_path, 'w') as f:
#         json.dump(mapping_collection, f)    

In [12]:
# df_path = os.path.join(POPULATION_OUT_ROOT, 'population_data.pkl')
output_dir = '/u/ayushc/projects/GradABM/MacroEcon/simulator_data/census_populations/NYC/{}_simulation_input/'.format(area_selector[0])

output_dir

'/u/ayushc/projects/GradABM/MacroEcon/simulator_data/census_populations/NYC/BK0101_simulation_input/'

In [ ]:
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
    
# preprocess_data(df_path, output_dir)

In [14]:
loader.export(output_dir)